In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures

pd.set_option('display.max_columns', 300)

## Step 1: Read in hold out data, scalers, and best model

In [2]:
df = pd.read_csv('Resources/kc_house_data_test_features.csv', index_col=0) #holdout
final_model = pickle.load(open('poly2_model.pickle','rb'))

In [3]:
print('Number of columns: ', len(final_model.coef_))

Number of columns:  28


## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [4]:
#changing date format workable month/year
df['yr_sold'] = pd.to_datetime(df['date'].str.slice(0,8), format='%Y%m%d', errors='ignore').dt.year
df['month_sold'] = pd.to_datetime(df['date'].str.slice(0,8), format='%Y%m%d', errors='ignore').dt.month
df.drop('date',axis=1,inplace=True)

In [5]:
#adjusting bedrooms outliers 
df['bedrooms'] = df['bedrooms'].apply(lambda x : 9 if x > 9 else x)

# adjuting sqft home outliers to within 6 standard deviations of the mean
for col in ['sqft_above', 'sqft_living', 'sqft_living15']: 
    above_6std = df[col].mean()+(6*df[col].std())
    df[col] = np.where(df[col].values >above_6std, df[col].mean()+6*df[col].std(), df[col])
    
# adjusting lot outliers to within 9 standard deviations of the mean
for col in ['sqft_lot', 'sqft_lot15']: 
    above_9std = df[col].mean()+(9*df[col].std())
    df[col] = np.where(df[col].values >above_6std, df[col].mean()+9*df[col].std(), df[col])
    

In [6]:
#lat/long cat?

In [7]:
## RUN THIS CELL ONLY ONCE!

#Create new column 'age' for age of house
df['age'] = 2020 - df.yr_built

#Create dummy variable 'basement' y/n
df['basement'] = np.where(df['sqft_basement']>0 , 1, 0)
df = pd.get_dummies(df, columns=['basement'])
df = df.drop(['basement_0'], axis=1)

#Create a new column 'neighbors_compared' with sqft_living - sqft_living15
df['neighbors_compared'] = df.sqft_living - df.sqft_living15

#Create dummy columns using 'nc_2'
df['nc2'] = np.where(df['sqft_living']> df['sqft_living15'] , 1, 0)
df = pd.get_dummies(df, columns=['nc2'])
df = df.drop(['nc2_0'], axis=1)

In [8]:
## RUN THIS CELL ONLY ONCE!

#Renovated columns
#Create new column 'last_renovted' for how long ago since last renovation
df['last_ren'] = np.where(df['yr_renovated'] > 0, (2020 - df['yr_renovated']), 0) ##np.nan may break this code... find out a way to make '0' into null value

#Create new column 'last_ren2' for how long (years) since last renovation and including age if no renovation
df['last_ren2'] = np.where(df['yr_renovated'] > 0, (2020 - df['yr_renovated']), (2020 - df['yr_built']))

#Create dummy columns using 'renovated'
df['renovated'] = np.where(df['yr_renovated']>0 , 1, 0)
df = pd.get_dummies(df, columns=['renovated'])
df = df.drop(['renovated_0'], axis=1)


In [9]:
# #Dummy Zips
# # Create dummy variables for zipcodes
# zip_dummies = pd.get_dummies(df['zipcode'], drop_first=True)

## Step 3: Predict the holdout set

In [10]:
df.columns

Index(['id', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'yr_sold', 'month_sold', 'age',
       'basement_1', 'neighbors_compared', 'nc2_1', 'last_ren', 'last_ren2',
       'renovated_1'],
      dtype='object')

In [11]:
poly= PolynomialFeatures(degree=2, include_bias=False)
poly_data = poly.fit_transform(df)
poly_columns = poly.get_feature_names(df.columns)
df_poly = pd.DataFrame(poly_data, columns=poly_columns)

In [12]:
df_poly.shape

(4323, 434)

In [13]:
df.shape

(4323, 28)

In [14]:
selected_columns = ['sqft_living',
 'grade',
 'bedrooms sqft_living',
 'bathrooms sqft_living',
 'bathrooms grade',
 'bathrooms sqft_above',
 'bathrooms sqft_living15',
 'sqft_living^2',
 'sqft_living floors',
 'sqft_living condition',
 'sqft_living grade',
 'sqft_living sqft_above',
 'sqft_living yr_built',
 'sqft_living lat',
 'sqft_living long',
 'sqft_living sqft_living15',
 'sqft_living yr_sold',
 'condition sqft_above',
 'grade^2',
 'grade sqft_above',
 'grade yr_built',
 'grade lat',
 'grade long',
 'grade sqft_living15',
 'grade yr_sold',
 'sqft_above^2',
 'sqft_above lat',
 'sqft_above sqft_living15']

In [15]:
final_pred = final_model.predict(df_poly[selected_columns])

In [16]:
final_pred.mean()

548751.5221319428

## Step 4: Export your predictions

In [17]:
last_df = pd.DataFrame(final_pred)

In [18]:
last_df.to_csv("housing_preds_Candaur.csv'")